In [2]:
import tensorflow as tf 
import numpy as np 
import matplotlib as mp
mp.use('TkAgg')
import matplotlib.pyplot as plt 
import os as os
import pandas as pd
from ML_tools import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from sklearn.preprocessing import StandardScaler

2023-08-10 15:11:03.713913: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 15:11:03.746636: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 15:11:03.747170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 15:11:04.320756: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
spark = SparkSession.builder \
        .appName('Company_Project') \
        .config("spark.jars", "mariadb-java-client-3.1.4.jar")\
        .getOrCreate()



23/08/10 15:11:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:

DOCS = spark.read.format("jdbc")\
        .option("url","jdbc:mariadb://localhost:3306/lnd")\
        .option("driver", "org.mariadb.jdbc.Driver")\
        .option("dbtable", "DOCS")\
        .option("user", "ETL")\
        .option("password", os.environ.get('PASS') )\
        .load()

In [5]:
w = Window().partitionBy().orderBy(col('Date'))
DOCS_ML = DOCS.withColumn("PrevClose", lag("close", 1, 0).over(w)) \
        .withColumn("Return", (col("close") - col("PrevClose"))/ col("PrevClose")) \
        .withColumn("Date", DOCS.Date.cast(DateType()))
DOCS_pd = DOCS_ML.toPandas()

23/08/10 15:11:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 15:11:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 15:11:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 15:11:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 15:11:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [6]:
# take return into a 1 dimentional numpy ary
series = DOCS_pd['Return'].values[1:].reshape(-1, 1)

# normalise data
scaler = StandardScaler()
scaler.fit(series[:len(series // 2)])
series = scaler.transform(series).flatten()

In [12]:
# Setup the data sets 

# N = number of samples in the data set
# T = Sequence length
# D =number of featurs
# M = number of hidden units
# K = number of output units

T = 10 
# T is the number of previous steps we will use to calculate the next 
D = 1
# D is 1 as we are only using 1 feature in this case Return 
X = []
Y = [] 

for t in range(len(series) - T):
    x = series[t:t+T]
    X.append(x)
    y = series[t+T]
    Y.append(y)

X = np.array(X).reshape(-1, T, 1) #now data should be N x T x D
Y = np.array(Y)
N = len(X)

In [15]:
print(X.shape, Y.shape)

(524, 10, 1) (524,)


In [8]:
# Create the Model 

i = tf.keras.layers.Input(shape=(T,1))
x = tf.keras.layers.LSTM(5)(i)
o = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(i, o)

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.01))

2023-08-10 15:11:10.886480: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-10 15:11:10.886927: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
# Train the Model 

r = model.fit(X[:-N//2], Y[:-N//2], 
              epochs = 90,
              validation_data=(X[-N//2:], Y[N//2:]),
              )

Epoch 1/90


9/9 [==============================] - 1s 37ms/step - loss: 1.2992 - val_loss: 0.6791
Epoch 2/90
9/9 [==============================] - 0s 6ms/step - loss: 1.2969 - val_loss: 0.6789
Epoch 3/90
9/9 [==============================] - 0s 6ms/step - loss: 1.2964 - val_loss: 0.6790
Epoch 4/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2961 - val_loss: 0.6789
Epoch 5/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2958 - val_loss: 0.6786
Epoch 6/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2956 - val_loss: 0.6789
Epoch 7/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2953 - val_loss: 0.6791
Epoch 8/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2949 - val_loss: 0.6786
Epoch 9/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2948 - val_loss: 0.6785
Epoch 10/90
9/9 [==============================] - 0s 5ms/step - loss: 1.2941 - val_loss: 0.6789
Epoch 11/90
9/9 [==============================] 

In [10]:
# multi step forcast 
validation_target = Y[N//2:]
validation_predictions = []

# last train input 
last_x = X[-N//2]

while len (validation_predictions) < len(validation_target):
#     1x1 array -> scalar
    p = model.predict(last_x.reshape(1, T, 1))[0,0]
    # update the prediction list
    validation_predictions.append(p)
    # make the new input 
    last_x = np.roll(last_x, -1)
    last_x[-1] = p 

1/1 [==============================] - 0s 11ms/step


In [11]:
plt.plot(validation_target, label='forcast target')
plt.plot(validation_predictions, label='forcast prediction')
plt.legend()
plt.show()
        